In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import PIL

import bg_atlasapi as bga

In [ ]:
import flexiznam as flz
from flexiznam.schema import Dataset
from brisc.example_sections import coronal_view_phpeb as cvp
import cricksaw_analysis.io

In [ ]:
# Define example data
project = "rabies_barcoding"
plane_per_section = 5
pixel_size = 2
mouse_filter = dict(Zresolution=8)
plot_prop = {
    "BRJN101.5c": dict(
        ENT=dict(xlim=[4250, 5200], ylim=[3050, 1550], z=350),
        PIR=dict(xlim=(3550, 4350), ylim=(2900, 2070), z=1180),
        AON=dict(xlim=(2750, 4070), ylim=(2980, 1900), z=1240),
        COA=dict(xlim=[3700, 4500], ylim=[3680, 3200], z=650),
    ),
    "BRJN101.5d": dict(
        ENT=dict(xlim=[3900, 5200], ylim=[3000, 1700], z=330),
        PIR=dict(xlim=(3400, 4300), ylim=(3000, 2200), z=1130),
        AON=dict(xlim=(2750, 3900), ylim=(2980, 2000), z=1200),
        COA=dict(xlim=[3700, 4400], ylim=[3580, 3000], z=570),
    ),
}
areas = ["VISp"]
atlas_size = 25

PROJECTION_FUNC = np.max  # must have a axis argument

In [ ]:
# Get atlas and flexilims info
atlas_name = "allen_mouse_%dum" % atlas_size
bg_atlas = bga.bg_atlas.BrainGlobeAtlas(atlas_name)

raw = Path(flz.PARAMETERS["data_root"]["raw"])
processed = Path(flz.PARAMETERS["data_root"]["processed"])
flm_sess = flz.get_flexilims_session(project_id=project)
mice = flz.get_entities(datatype="mouse", flexilims_session=flm_sess)
for filt, val in mouse_filter.items():
    mice = mice[mice[filt] == val].copy()

In [ ]:
import cricksaw_analysis.cricksaw_to_flexilims

for mn, mouse in mice.iterrows():
    print("Processing %s" % mouse.name)
    cellfinder_folder = processed / project / mouse.name / "cellfinder_results"
    if cellfinder_folder.exists():
        pts, downsampled_stacks, atlas = cricksaw_analysis.io.load_cellfinder_results(
            cellfinder_folder
        )
        logs = sorted(list(cellfinder_folder.glob("cellfinder_*.log")))
        try:
            ds = cricksaw_analysis.cricksaw_to_flexilims.create_dataset_from_log(
                logs[-1], parent_id=mouse.id, flexilims_session=flm_sess
            )
            ds.path = cellfinder_folder.relative_to(processed)
            ds.update_flexilims()
        except flz.errors.DatasetError:
            print("Dataset already exists")
            continue
    else:
        raise IOError("Cellfinder folder not found")

In [ ]:
add_detected_cells = False
handpicked_only = False
for _, mouse in mice.iterrows():
    fig = plt.figure()
    # look if I have cellfinder data
    # print("Doing %s" % mouse.name, flush=True)
    if handpicked_only and (mouse.name not in plot_prop):
        print("%s not in plot_prop, skip" % mouse.name)
        continue
    try:
        cellfinder_data, cells, atlas_coord, atlas_id = cvp.get_cell_data(
            mouse, flm_sess, bg_atlas
        )
    except IOError:
        print("No cellfinder data, skip")
        continue

    trans_atlas = cvp.load_in_brainsaw_coordinates(
        cellfinder_data.path_full / "registration" / "registered_atlas.tiff"
    )
    trans_hem = cvp.load_in_brainsaw_coordinates(
        cellfinder_data.path_full / "registration" / "registered_hemispheres.tiff"
    )

    # remove the left hemisphere to label only half the brain
    trans_atlas[trans_hem == 1] = 0
    # keep only the right hemisphere of detected cells
    midline = bg_atlas.annotation.shape[-1] / 2
    is_right = atlas_coord[:, 2] < midline

    ap_axis, dv_axis, ml_axis = (2, 1, 0)

    fig.clear()
    fig.suptitle(f"Mouse {mouse.name}\n{mouse.Dilution}")
    raw_data_dir = raw / project / mouse.name / "stitchedImages_100"
    assert raw_data_dir.is_dir()

    o = cvp.get_cricksaw_metadata(raw_data_dir)
    if o is None:
        print(f"{mouse.name} still archived, skip!!!")
        continue

    for iax, parent_area in enumerate(areas):
        ax = fig.add_subplot(1, 1, iax + 1)
        cvp.plot_area(
            ax,
            raw_data_dir,
            area=parent_area,
            bg_atlas=bg_atlas,
            atlas_id=atlas_id[is_right],
            cells=cells[is_right],
            mouse_prop=plot_prop.get(mouse.name, {}),
            atlas_volume=trans_atlas,
            projection_func=np.max,
            add_detected_cells=False,
        )
        ax.set_axis_on()

    # fig.subplots_adjust(top=0.95, bottom=0, left=0, right=1, wspace=0, hspace=0.05)
    # save output
    ds = Dataset.from_origin(
        origin_id=mouse.id,
        flexilims_session=flm_sess,
        dataset_type="microscopy",
        conflicts="append",
    )
    ds.flexilims_session = flm_sess
    ds.genealogy = list(ds.genealogy)[:-1] + ["coronal_view_%s.svg" % ("_".join(areas))]
    ds.extra_attributes["atlas"] = atlas_name
    ds.extra_attributes["num_slices"] = plane_per_section
    ds.extra_attributes["projection"] = repr(PROJECTION_FUNC)
    ds.extra_attributes["pixel_size"] = pixel_size
    ds.path = ds.path.parent / ds.genealogy[-1]
    fig.savefig(ds.path_full, dpi=1200)
    print(f"{mouse.name} done.")
    ds.update_flexilims(mode="overwrite")
    # plt.xlim(500, 1324)
    # plt.ylim(400, 0)